In [1]:
import pandas as pd
import numpy as np
import json
import plotly.express as px

In [2]:
player_info = pd.read_csv("player_stats/player_basic_info.csv", index_col = 0)
with open("player_moves/trade_list_2020s.json") as f:
    trade_info = json.load(f)

In [3]:
df1 = pd.DataFrame(list(map(lambda x : [x["id"], x["date"], x["teamA"], x["teamB"]], trade_info)),
    columns=["id", "date", "teamA", "teamB"])
df1["date"] = pd.to_datetime(df1["date"])

df1_0 = df1.pivot_table(df1, index=df1["date"].apply(lambda x : x.year), aggfunc="count")["id"]
fig1_0 = px.bar(df1_0, x=df1_0.index, y="id",
    labels=dict(date="Year", id="count"))
fig1_0.update_yaxes(range=[4, 9.5])

In [4]:
df1_1 = df1[df1["date"].map(lambda x : x.year) == 2020]
fig1_1 = px.bar(df1_1["teamA"])

In [5]:
trade_list = []
for trade in trade_info:
    for x in trade["playerA"]:
        if x["type"] == "player":
            trade_list.append([trade["date"], x["name"], x["statizId"], trade["teamA"], trade["teamB"]])
    for x in trade["playerB"]:
        if x["type"] == "player":
            trade_list.append([trade["date"], x["name"], x["statizId"], trade["teamB"], trade["teamA"]])

df2 = pd.DataFrame(trade_list, columns=["date", "name", "statizId", "원소속팀", "이적팀"])
df2["date"] = pd.to_datetime(df2["date"])
player_info["statizId"] = player_info["statizId"].astype(str)
df2 = df2.merge(player_info, on="statizId", how="left")[["date", "name", "statizId", "원소속팀", "이적팀", "주포지션"]]

In [6]:
player_info = pd.read_csv("player_stats/player_basic_info.csv", index_col = 0)
with open("player_moves/trade_list_2020s.json", encoding='UTF8') as f:
    trade_info = json.load(f)

trade_list = []
for trade in trade_info:
    for x in trade["playerA"]:
        if x["type"] == "player":
            trade_list.append([trade["date"], x["name"], x["statizId"], trade["teamA"], trade["teamB"]])
    for x in trade["playerB"]:
        if x["type"] == "player":
            trade_list.append([trade["date"], x["name"], x["statizId"], trade["teamB"], trade["teamA"]])

df2 = pd.DataFrame(trade_list, columns=["date", "name", "statizId", "원소속팀", "이적팀"])
df2["date"] = pd.to_datetime(df2["date"])
player_info["statizId"] = player_info["statizId"].astype(str)
df2 = df2.merge(player_info, on="statizId", how="left")[["date", "name", "statizId", "원소속팀", "이적팀", "주포지션"]]

selected_year = 2020
df1_2 = df2[df2["date"].map(lambda x : x.year) == selected_year].reset_index(drop=True)
infielder = ["C", "1B", "2B", "3B", "4B", "SS"]
outfielder = ["LF", "RF", "CF"]

df1_2["주포지션_내외야"] = \
    pd.Series(["Infielder" if x in infielder else "outfielder" if x in outfielder else "None" for x in df1_2["주포지션"]])

pd.pivot_table(df1_2, index=["주포지션_내외야", "주포지션"], aggfunc="count")
df1_2 = df1_2.pivot_table(index=["주포지션_내외야", "주포지션"], aggfunc="count")["statizId"].reset_index()

df1_2.loc[df1_2["주포지션_내외야"] == "None", "주포지션"] = None
df1_2.loc[df1_2["주포지션_내외야"] == "None", "주포지션_내외야"] = "P"

fig1_2 = px.sunburst(df1_2, path=["주포지션_내외야", "주포지션"], values="statizId")
fig1_2.update_layout(height=400, margin=dict(l=20, r=20, t=20, b=20))


In [7]:
from assets.dataframe import df1, df2

df1 = df1()
df2 = df2()

In [8]:
import pandas as pd
import plotly.graph_objects as go

label_list = ['cat', 'dog', 'domesticated', 'female', 'male', 'wild']
# cat: 0, dog: 1, domesticated: 2, female: 3, male: 4, wild: 5
source = [0, 0, 1, 3, 4, 4]
target = [3, 4, 4, 2, 2, 5]
count = [21, 6, 22, 21, 6, 22]

fig = go.Figure(data=[go.Sankey(
    node = {"label": label_list},
    link = {"source": source, "target": target, "value": count}
    )])
fig.show()

In [9]:
temp = pd.DataFrame(np.vstack([df1[["teamA", "teamB"]], df1[["teamB", "teamA"]]]), columns=["col1", "col2"])
matrix= np.array(pd.crosstab(temp["col1"], temp["col2"]), dtype=int)

In [83]:
trade_cases = []

for idx, trade in enumerate(trade_info):
    for a in trade["playerA"]:
        if a["type"] == "player":
            trade_cases.append([trade["id"], trade["date"], trade["teamA"], trade["teamB"], a["type"], a["name"]])
        elif a["type"] == "draft":
            trade_cases.append([trade["id"], trade["date"], trade["teamA"], trade["teamB"], a["type"], a["round"]])
        elif a["type"] == "money":
            trade_cases.append([trade["id"], trade["date"], trade["teamA"], trade["teamB"], a["type"], a["amount"]])
    
    for b in trade["playerB"]:
        if b["type"] == "player":
            trade_cases.append([trade["id"], trade["date"], trade["teamB"], trade["teamA"], b["type"], b["name"]])
        elif b["type"] == "draft":
            trade_cases.append([trade["id"], trade["date"], trade["teamB"], trade["teamA"], b["type"], b["round"]])
        elif b["type"] == "money":
            trade_cases.append([trade["id"], trade["date"], trade["teamB"], trade["teamA"], b["type"], b["amount"]])

df3 = pd.DataFrame(trade_cases, columns=["id", "date", "원소속팀", "이적팀", "트레이드타입", "트레이드자원"])
freq = df3[["원소속팀", "이적팀"]].value_counts().to_frame().reset_index()
df3 = df3.merge(freq, on=["원소속팀", "이적팀"])
df3 = df3[["원소속팀", "이적팀", "count"]].drop_duplicates().reset_index(drop=True)
team_labels = list(df3["원소속팀"].unique())

df3["원소속팀_code"] = df3["원소속팀"].apply(lambda x : f"source_{x}")
df3["이적팀_code"] = df3["이적팀"].apply(lambda x : f"dest_{x}")

In [95]:
label = list(np.unique(np.array(df3[["원소속팀_code", "이적팀_code"]]).ravel()))
label

['dest_KIA',
 'dest_KT',
 'dest_LG',
 'dest_NC',
 'dest_SSG',
 'dest_두산',
 'dest_롯데',
 'dest_삼성',
 'dest_키움',
 'dest_한화',
 'source_KIA',
 'source_KT',
 'source_LG',
 'source_NC',
 'source_SSG',
 'source_두산',
 'source_롯데',
 'source_삼성',
 'source_키움',
 'source_한화']

In [102]:
source = list(df3["원소속팀_code"].apply(lambda x : label.index(x)))
dest = list(df3["이적팀_code"].apply(lambda x : label.index(x)))

In [125]:
import plotly.graph_objects as go

node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label
    )
  
link = dict(
      source = source,
      target = dest,
      value = df3["count"]
  )


fig = go.Figure(data=[go.Sankey(
    node = node,
    link = link
    )])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=15)
fig.show()

In [156]:
trade_cases = []

for idx, trade in enumerate(trade_info):
    for a in trade["playerA"]:
        if a["type"] == "player":
            trade_cases.append([trade["id"], trade["date"], trade["teamA"], trade["teamB"], a["type"], a["name"]])
        elif a["type"] == "draft":
            trade_cases.append([trade["id"], trade["date"], trade["teamA"], trade["teamB"], a["type"], a["round"]])
        elif a["type"] == "money":
            trade_cases.append([trade["id"], trade["date"], trade["teamA"], trade["teamB"], a["type"], a["amount"]])
    
    for b in trade["playerB"]:
        if b["type"] == "player":
            trade_cases.append([trade["id"], trade["date"], trade["teamB"], trade["teamA"], b["type"], b["name"]])
        elif b["type"] == "draft":
            trade_cases.append([trade["id"], trade["date"], trade["teamB"], trade["teamA"], b["type"], b["round"]])
        elif b["type"] == "money":
            trade_cases.append([trade["id"], trade["date"], trade["teamB"], trade["teamA"], b["type"], b["amount"]])

df3 = pd.DataFrame(trade_cases, columns=["id", "date", "원소속팀", "이적팀", "트레이드타입", "트레이드자원"])
df3.head()

,id,date,원소속팀,이적팀,트레이드타입,트레이드자원
0,1,2020-01-28,키움,KIA,player,장영석
1,1,2020-01-28,KIA,키움,player,박준태
2,1,2020-01-28,KIA,키움,money,2억원
3,2,2020-04-06,키움,롯데,player,추재현
4,2,2020-04-06,롯데,키움,player,전병우


In [168]:
df3_1 = pd.pivot_table(df3[["원소속팀", "이적팀", "id"]], values="id", index="원소속팀", columns="이적팀", aggfunc="count")
df3_1

이적팀,KIA,KT,LG,NC,SSG,두산,롯데,삼성,키움,한화
원소속팀,,,,,,,,,,
KIA,NaN,NaN,NaN,2.0,1.0,1.0,NaN,1.0,6.0,5.0
KT,NaN,NaN,1.0,NaN,2.0,NaN,4.0,1.0,NaN,1.0
LG,NaN,1.0,NaN,2.0,NaN,2.0,3.0,NaN,4.0,NaN
NC,2.0,NaN,2.0,NaN,1.0,NaN,1.0,1.0,2.0,2.0
SSG,2.0,2.0,NaN,2.0,NaN,3.0,NaN,NaN,4.0,1.0
두산,1.0,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN
롯데,NaN,5.0,3.0,1.0,NaN,NaN,NaN,2.0,2.0,NaN
삼성,1.0,1.0,NaN,2.0,NaN,NaN,1.0,NaN,2.0,1.0
키움,3.0,NaN,2.0,1.0,2.0,NaN,1.0,1.0,NaN,NaN


In [175]:
fig = px.imshow(df3_1, text_auto=True, aspect="auto", color_continuous_scale=px.colors.sequential.Viridis)
fig.show()

In [195]:
df4 = df3.loc[df3["원소속팀"] == "KIA", ["date", "원소속팀", "이적팀", "트레이드타입", "트레이드자원"]]

In [194]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')
df.to_dict('records')[:1]

[{'State': 'California',
  'Number of Solar Plants': 289,
  'Installed Capacity (MW)': 4395,
  'Average MW Per Plant': 15.3,
  'Generation (GWh)': 10826}]

In [196]:
df4.to_dict('records')

[{'date': '2020-01-28',
  '원소속팀': 'KIA',
  '이적팀': '키움',
  '트레이드타입': 'player',
  '트레이드자원': '박준태'},
 {'date': '2020-01-28',
  '원소속팀': 'KIA',
  '이적팀': '키움',
  '트레이드타입': 'money',
  '트레이드자원': '2억원'},
 {'date': '2020-06-07',
  '원소속팀': 'KIA',
  '이적팀': '두산',
  '트레이드타입': 'player',
  '트레이드자원': '홍건희'},
 {'date': '2020-08-12',
  '원소속팀': 'KIA',
  '이적팀': 'NC',
  '트레이드타입': 'player',
  '트레이드자원': '문경찬'},
 {'date': '2020-08-12',
  '원소속팀': 'KIA',
  '이적팀': 'NC',
  '트레이드타입': 'player',
  '트레이드자원': '박정수'},
 {'date': '2021-07-03',
  '원소속팀': 'KIA',
  '이적팀': '한화',
  '트레이드타입': 'player',
  '트레이드자원': '백용환'},
 {'date': '2022-04-23',
  '원소속팀': 'KIA',
  '이적팀': '한화',
  '트레이드타입': 'player',
  '트레이드자원': '이민우'},
 {'date': '2022-04-23',
  '원소속팀': 'KIA',
  '이적팀': '한화',
  '트레이드타입': 'player',
  '트레이드자원': '이진영'},
 {'date': '2022-04-24',
  '원소속팀': 'KIA',
  '이적팀': '키움',
  '트레이드타입': 'player',
  '트레이드자원': '김태진'},
 {'date': '2022-04-24',
  '원소속팀': 'KIA',
  '이적팀': '키움',
  '트레이드타입': 'money',
  '트레이드자원': '10억원'},
 {'date': '2022-04-24

In [222]:
df4.loc[(df4["원소속팀"] == "KIA") & (df4["이적팀"] == "키움"), :]

,date,원소속팀,이적팀,트레이드타입,트레이드자원
1,2020-01-28,KIA,키움,player,박준태
2,2020-01-28,KIA,키움,money,2억원
57,2022-04-24,KIA,키움,player,김태진
58,2022-04-24,KIA,키움,money,10억원
59,2022-04-24,KIA,키움,draft,23-2
73,2022-11-11,KIA,키움,draft,24-2
